# [과제 3] 로지스틱 회귀분석
### - sklearn 패키지를 사용해 로지스틱 회귀분석을 진행해주세요.
### - 성능지표를 계산하고 이에 대해 해석해주세요.
### - 성능 개선을 시도해주세요. (어떠한 성능지표를 기준으로 개선을 시도했는지, 그 이유도 함께 적어주세요.)
### - 주석으로 설명 및 근거 자세하게 달아주시면 감사하겠습니다. :)

## Data 

출처 : https://www.kaggle.com/mlg-ulb/creditcardfraud


* V1 ~ V28 : 비식별화 된 개인정보 
* **Class** : Target 변수  
  - 1 : fraudulent transactions (사기)
  - 0 : otherwise 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [20]:
data = pd.read_csv("assignment3_creditcard.csv")

In [21]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,-1.848212,2.384900,0.379573,1.048381,-0.845070,2.537837,-4.542983,-10.201458,-1.504967,-2.234167,...,2.585817,-5.291690,0.859364,0.423231,-0.506985,1.020052,-0.627751,-0.017753,0.280982,0
1,2.071805,-0.477943,-1.444444,-0.548657,0.010036,-0.582242,-0.042878,-0.247160,1.171923,-0.342382,...,-0.077306,0.042858,0.390125,0.041569,0.598427,0.098803,0.979686,-0.093244,-0.065615,0
2,-2.985294,-2.747472,1.194068,-0.003036,-1.151041,-0.263559,0.553500,0.635600,0.438545,-1.806488,...,1.345776,0.373760,-0.385777,1.197596,0.407229,0.008013,0.762362,-0.299024,-0.303929,0
3,-1.479452,1.542874,0.290895,0.838142,-0.529290,-0.717661,0.484516,0.545092,-0.780767,0.324804,...,0.038397,0.116771,0.405560,-0.116453,0.541275,-0.216665,-0.415578,0.027126,-0.150347,0
4,-0.281976,-0.309699,-2.162299,-0.851514,0.106167,-1.483888,1.930994,-0.843049,-1.249272,1.079608,...,-0.875516,-0.004199,1.015108,-0.026748,0.077115,-1.468822,0.751700,0.496732,0.331001,0


In [22]:
target = pd.DataFrame(data['Class'])
target

,Class
0,0
1,0
2,0
3,0
4,0
...,...
28673,0
28674,0
28675,0
28676,0


In [23]:
data.drop(columns='Class',inplace=True)

In [24]:
# feature 만 있는 상태로 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit_transform(data)

array([[-0.90218019,  1.44246214,  0.24207645, ..., -1.29584288,
        -0.04709654,  0.9417547 ],
       [ 1.02744235, -0.32892601, -0.7953942 , ...,  2.02637926,
        -0.23721911, -0.22629752],
       [-1.46190736, -1.73320058,  0.7053479 , ...,  1.57721728,
        -0.7554671 , -1.02943014],
       ...,
       [ 1.02786291,  0.0843943 , -0.9747001 , ...,  0.36917223,
        -0.15087861, -0.1030219 ],
       [-0.33236542,  0.79733419,  0.8782926 , ...,  0.21096583,
        -2.140772  , -1.05999649],
       [ 1.09831537, -0.52987507, -0.90637085, ..., -0.61547609,
        -0.13776702, -0.2565925 ]])

In [25]:
data = pd.DataFrame(scaler.fit_transform(data), columns = data.columns)

In [26]:
data

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
0,-0.902180,1.442462,0.242076,0.682170,-0.545636,1.897453,-3.087657,-7.481774,-1.310739,-1.750813,...,0.184394,3.586292,-6.258702,1.163043,0.721013,-0.838813,1.973207,-1.295843,-0.047097,0.941755
1,1.027442,-0.328926,-0.795394,-0.395393,0.029429,-0.432294,0.005620,-0.185605,1.049741,-0.240491,...,0.770828,-0.110982,0.038445,0.528470,0.078108,0.993348,0.194594,2.026379,-0.237219,-0.226298
2,-1.461907,-1.733201,0.705348,-0.027248,-0.751404,-0.194335,0.415558,0.461429,0.403048,-1.409372,...,-1.078858,1.864714,0.429058,-0.520817,2.025422,0.676446,0.019311,1.577217,-0.755467,-1.029430
3,-0.720659,0.921458,0.191638,0.540317,-0.333272,-0.533410,0.368140,0.395090,-0.672139,0.292162,...,0.793099,0.049651,0.125696,0.549344,-0.188079,0.898622,-0.414465,-0.857327,0.065929,-0.511850
4,-0.131203,-0.224824,-1.203698,-0.599739,0.094078,-1.105548,1.362418,-0.622372,-1.085267,0.894765,...,-0.651400,-1.219154,-0.017103,1.373663,-0.036971,0.129301,-2.831946,1.555181,1.248615,1.110322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28673,1.023255,-0.023908,-0.566583,0.251205,-0.024285,-0.892442,0.202475,-0.286709,0.376723,0.071062,...,0.156931,-0.239323,-0.341422,-0.845179,0.566116,0.117848,-0.517101,0.399455,-0.165871,-0.200830
28674,0.630217,-0.793406,0.924346,-0.163191,-1.294682,0.410528,-1.177492,0.340326,0.900087,0.251606,...,0.769189,-0.177753,0.158912,0.972702,-0.056755,0.173374,0.662386,-0.018687,0.223756,0.061716
28675,1.027863,0.084394,-0.974700,0.230460,0.397149,-0.572397,0.204807,-0.205689,0.352270,-0.290601,...,0.117016,-0.120511,-0.438509,-1.230805,0.520796,0.666969,-0.406308,0.369172,-0.150879,-0.103022
28676,-0.332365,0.797334,0.878293,1.357837,0.732640,0.508122,0.623108,-0.346918,-0.584673,1.218846,...,0.060428,0.039199,0.118811,0.613586,-0.590491,-1.276522,-0.688309,0.210966,-2.140772,-1.059996


In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# train, test 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state = 11, stratify = target)

In [28]:
# 로지스틱 리그레션
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [30]:
#예측
y_pred = model.predict(X_test) 
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [37]:
from sklearn.metrics import *
#평가지표들

In [38]:
confusion_matrix(y_test, y_pred)

array([[5683,    4],
       [   7,   42]], dtype=int64)

### confusion matrix 결과 클래스 비율 차이 너무 큼

In [39]:
precision_score(y_test, y_pred)

0.9130434782608695

In [40]:
recall_score(y_test, y_pred)

0.8571428571428571

In [41]:
f1_score(y_pred, y_test)

0.8842105263157894

### 오버 샘플링
#### 이용한 이유 : 클래스 간의 비율 차이가 너무 커서 오버 샘플링을 통해 f1-score, recall, precision을 향상시켜보고자 함

In [44]:
X = data
y = target

In [47]:
! pip install imblearn

     -------------------------------------- 226.0/226.0 kB 7.0 MB/s eta 0:00:00
     -------------------------------------- 298.0/298.0 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0



[notice] A new release of pip available: 22.3 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
# SMOTE 방식으로 오버 샘플링함
import imblearn
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=0).fit_resample(X, y)
X_resampled = pd.DataFrame(X_resampled)
y_resampled = pd.DataFrame(y_resampled)

In [61]:
X_resampled,y_resampled

(             V1        V2        V3        V4        V5        V6        V7  \
 0     -0.902180  1.442462  0.242076  0.682170 -0.545636  1.897453 -3.087657   
 1      1.027442 -0.328926 -0.795394 -0.395393  0.029429 -0.432294  0.005620   
 2     -1.461907 -1.733201  0.705348 -0.027248 -0.751404 -0.194335  0.415558   
 3     -0.720659  0.921458  0.191638  0.540317 -0.333272 -0.533410  0.368140   
 4     -0.131203 -0.224824 -1.203698 -0.599739  0.094078 -1.105548  1.362418   
 ...         ...       ...       ...       ...       ...       ...       ...   
 56859 -0.538488  1.934090 -3.696315  3.351348 -1.538580 -1.684775 -3.798950   
 56860 -3.475956  1.991299 -5.228885  3.522013 -2.568718 -2.020284 -4.949708   
 56861 -1.762500  1.441722 -2.664643  1.088216 -0.794260 -1.279955 -1.943718   
 56862  0.135086  1.498990 -2.473874  2.871097 -0.391440 -1.055657 -1.969293   
 56863 -0.839456  1.308949 -3.898164  3.524809 -1.748492 -1.599731 -3.510272   
 
              V8        V9       V10  

In [62]:
# 앞의 절차와 동일함
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state = 11, stratify = y_resampled)
model = LogisticRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test) 

confusion_matrix(y_test, y_pred)

array([[5532,  154],
       [ 366, 5321]], dtype=int64)

In [63]:
precision_score(y_test, y_pred)

0.9718721461187214

In [64]:
recall_score(y_test, y_pred)

0.9356426938631968

In [65]:
f1_score(y_pred, y_test)

0.9534133667801469

### 전반적으로 성능 향상됨을 확인할 수 있다